In [ ]:
# 相關套件
import os
import ast
import logging # [優化點] 1. 改用 logging 模組
import traceback
import pandas as pd
import networkx as nx
from pyvis.network import Network
from datetime import datetime
from collections import Counter, defaultdict
from typing import Dict, Any, List, Optional, Tuple

# 處理可選套件
try:
    from pyecharts.charts import Tree
    from pyecharts import options as opts
    PYECHARTS_INSTALLED = True
except ImportError:
    PYECHARTS_INSTALLED = False

# --- 0. 設定區 ---
class Config:
    """集中管理所有設定"""
    JOB_CODE_PREFIX = '2007'
    JOB_LEVELS = {'2007001': '軟體/工程類人員', '2007002': 'MIS/網管類人員'}
    TODAY_STR = datetime.now().strftime('%Y%m%d')
    FILE_DATE = datetime.now().strftime("%Y%m%d")
    FILE_PATH = f"104人力銀行_職業探索_({FILE_DATE}).csv"
    
    
    # [優化點] 2. 統一實體設定，提高內聚性與可擴展性
    # 新增實體時，只需在此處新增一個字典即可
    ENTITY_CONFIG = {
        'Skill': {'list_col': 'hardSkillList', 'min_count': 1, 'color': '#ff7f0e', 'base_size': 12, 'size_multiplier': 1.5},
        'Tool':  {'list_col': 'hardToolList',  'min_count': 1, 'color': '#1f77b4', 'base_size': 12, 'size_multiplier': 1.5},
        # 若未來有 'Certificate'，可直接在此新增
        # 'Certificate': {'list_col': 'certificateList', 'min_count': 1, 'color': '#2ca02c', 'base_size': 10, 'size_multiplier': 2.0},
    }

    ROOT_NODE_ID = "所有軟體職位"
    NODE_TYPES = {
        'Root':     {'color': "#C66E76", 'base_size': 35},
        'Category': {'color': '#FFEBF2', 'base_size': 28},
        'Job':      {'color': '#87CEFA', 'base_size': 22},
        **{k: {'color': v['color'], 'base_size': v['base_size']} for k, v in ENTITY_CONFIG.items()}
    }

    # 心智圖顏色設定
    MIND_MAP_COLORS = {
        "root": NODE_TYPES['Root']['color'],
        "categories": {'軟體/工程類人員': "#B22222", 'MIS/網管類人員': "#4682B4"},
        **{k.lower(): v['color'] for k, v in ENTITY_CONFIG.items()}
    }

    # 輸出檔案名稱
    NETWORK_FILENAME = f"job_skill_network_{TODAY_STR}.html"
    MIND_MAP_FILENAME = f"job_skill_mind_map_{TODAY_STR}.html"
    LOG_FILENAME = f"job_analyzer_{TODAY_STR}.log"

    # UI 字串
    TITLE_STRINGS = {
        'entity_hover': "{type}: {name}\n--------------------\n相關職位:\n{jobs}",
        'job_hover': "職位: {name}\n--------------------\n描述: {summary}",
        'category_hover': "分類: {name}",
        'no_summary': "無摘要"
    }

# [優化點] 1. 設定 logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(Config.LOG_FILENAME, 'w', 'utf-8'),
        logging.StreamHandler()
    ]
)

# --- 1. 資料讀取與轉換 ---
class DataProcessor:
    """職責：讀取、驗證、清洗並轉換原始資料。"""
    def __init__(self, config: Config):
        self.config = config

    # [優化點] 3. 將輔助函式提升為靜態方法，增加程式碼純淨度
    @staticmethod
    def _safe_eval(cell_value: Any) -> List[Dict]:
        """安全地將字串轉換為列表，處理空值或格式錯誤。"""
        if pd.isna(cell_value) or cell_value in ['[]', '']:
            return []
        try:
            return ast.literal_eval(cell_value)
        except (ValueError, SyntaxError):
            logging.warning(f"無法解析的儲存格內容：{cell_value[:50]}...")
            return []

    def process(self) -> Optional[pd.DataFrame]:
        """執行完整的資料處理流程。"""
        file_path = self.config.FILE_PATH
        if not os.path.exists(file_path):
            logging.error(f"檔案不存在 -> {file_path}")
            return None
        
        logging.info(f"正在讀取檔案：{file_path}")
        df = pd.read_csv(file_path)
        
        required_cols = ['jobCode', 'jobName', 'jobSummary'] + [v['list_col'] for v in self.config.ENTITY_CONFIG.values()]
        if not all(col in df.columns for col in required_cols):
            logging.error(f"CSV 檔案缺少必要欄位。需要：{required_cols}")
            return None

        for entity_config in self.config.ENTITY_CONFIG.values():
            list_col = entity_config['list_col']
            df[list_col] = df[list_col].apply(self._safe_eval)
        
        df['jobCode'] = df['jobCode'].astype(str)
        
        df_filtered = df[df['jobCode'].str.startswith(self.config.JOB_CODE_PREFIX, na=False)].copy()
        if df_filtered.empty:
            logging.warning(f"找不到任何以 '{self.config.JOB_CODE_PREFIX}' 開頭的資料。")
            return None

        logging.info("階段 1: 完成檔案資料讀取與預處理。")
        return df_filtered

# --- 2. NetworkX 圖製作 ---
class GraphBuilder:
    """職責：根據處理好的資料，建立 NetworkX 圖模型。"""
    def __init__(self, config: Config):
        self.config = config
        self.graph = nx.DiGraph()
        self.entity_data: Dict[str, Dict[str, Any]] = {}

    def build(self, df: pd.DataFrame) -> Tuple[nx.DiGraph, Dict]:
        """建立圖，並回傳圖物件和實體資料。"""
        self._prepare_entity_data(df)
        self._build_base_structure()
        self._populate_from_dataframe(df)
        logging.info("階段 2: 完成關係圖 Graph 建立。")
        return self.graph, self.entity_data

    def _prepare_entity_data(self, df: pd.DataFrame):
        """預先計算每個實體的出現次數和關聯職位"""
        for entity_type, config in self.config.ENTITY_CONFIG.items():
            list_col = config['list_col']
            all_items = [item['name'] for sublist in df[list_col] for item in sublist if 'name' in item]
            counts = Counter(all_items)
            
            job_map = defaultdict(list)
            for _, row in df.iterrows():
                for item in row.get(list_col, []):
                    if item.get('name'):
                        job_map[item['name']].append(row['jobName'])
            
            self.entity_data[entity_type] = {'counts': counts, 'job_map': job_map}

    def _add_node(self, node_id: str, node_type: str, **kwargs):
        """統一新增節點的方法，自動從 Config 讀取屬性"""
        if self.graph.has_node(node_id):
            return
        
        attrs = {
            'label': node_id,
            'type': node_type,
            'color': self.config.NODE_TYPES[node_type]['color'],
            'size': self.config.NODE_TYPES[node_type]['base_size'],
            'title': node_id,
            **kwargs
        }
        self.graph.add_node(node_id, **attrs)

    def _build_base_structure(self):
        """建立 Root 和 Category 層級的節點"""
        root_id = self.config.ROOT_NODE_ID
        self._add_node(root_id, 'Root', title='總覽')
        
        for code, name in self.config.JOB_LEVELS.items():
            self._add_node(name, 'Category', title=self.config.TITLE_STRINGS['category_hover'].format(name=name))
            self.graph.add_edge(root_id, name)

    def _populate_from_dataframe(self, df: pd.DataFrame):
        """從 DataFrame 填充 Job 和 Entity 節點"""
        for _, row in df.iterrows():
            job_name = row['jobName']
            summary = row.get('jobSummary', self.config.TITLE_STRINGS['no_summary'])
            
            try:
                job_code_prefix = row['jobCode'][:7]
            except (TypeError, IndexError):
                logging.warning(f"職位 '{job_name}' 的 jobCode '{row['jobCode']}' 格式不正確，跳過。")
                continue

            level_name = self.config.JOB_LEVELS.get(job_code_prefix)
            if not level_name:
                continue

            self._add_node(job_name, 'Job', title=summary)
            self.graph.add_edge(level_name, job_name)
            
            for entity_type, config in self.config.ENTITY_CONFIG.items():
                self._add_entity_nodes_and_edges(job_name, row, entity_type, config)

    def _add_entity_nodes_and_edges(self, job_name: str, row: pd.Series, entity_type: str, config: Dict):
        list_col = config['list_col']
        min_count = config['min_count']
        counts_map = self.entity_data[entity_type]['counts']
        
        for item in row.get(list_col, []):
            item_name = item.get('name')
            if item_name and counts_map.get(item_name, 0) >= min_count:
                # [優化點] 4. 將魔術數字 1.5 移入設定檔
                size_multiplier = config.get('size_multiplier', 1.0)
                dynamic_size = counts_map[item_name] * size_multiplier
                size = config['base_size'] + dynamic_size
                
                self._add_node(item_name, entity_type, size=size)
                self.graph.add_edge(job_name, item_name)

# --- 3. 視覺化圖表製作 ---
class ReportGenerator:
    """職責：根據圖模型，產生各種視覺化報告（HTML檔案）。"""
    def __init__(self, config: Config):
        self.config = config

    def generate_network_html(self, graph: nx.DiGraph, entity_data: Dict):
        if not graph or graph.number_of_nodes() <= 1:
            logging.warning("圖為空或節點太少，不產生網絡圖。")
            return
            
        net = Network(height="800px", width="100%", notebook=True, directed=True, bgcolor="#222222", font_color="white", cdn_resources='remote')
        
        for node_id, attrs in graph.nodes(data=True):
            title = attrs.get('title', node_id)
            node_type = attrs.get('type')

            # [優化點] 5. 使用 Config 中的字串模板
            if node_type in entity_data:
                job_map = entity_data[node_type]['job_map']
                jobs_html = "、".join(sorted(list(set(job_map.get(node_id, [])))))
                title = self.config.TITLE_STRINGS['entity_hover'].format(type=node_type, name=node_id, jobs=jobs_html)
            elif node_type == 'Job':
                 title = self.config.TITLE_STRINGS['job_hover'].format(name=node_id, summary=attrs.get('title', self.config.TITLE_STRINGS['no_summary']))
            
            net.add_node(n_id=node_id, label=attrs.get('label'), color=attrs.get('color'), size=attrs.get('size'), title=title)
        
        net.add_edges(graph.edges())
        
        try:
            net.write_html(self.config.NETWORK_FILENAME)
            logging.info(f"成功產生網絡圖檔案：{self.config.NETWORK_FILENAME}")
        except Exception as e:
            logging.error(f"產生網絡圖失敗: {e}")


    def _build_tree_data(self, graph: nx.DiGraph, node_id: str) -> Dict:
        """遞迴建構 Pyecharts Tree 所需的資料結構 (私有輔助方法)"""
        node_attrs = graph.nodes[node_id]
        node_type = node_attrs.get('type', '').lower()
        node_data: Dict[str, Any] = {"name": node_id}

        color_map = self.config.MIND_MAP_COLORS
        color = None
        if node_type == 'root':
            color = color_map.get("root")
        elif node_type == 'category':
            color = color_map["categories"].get(node_id)
        else:
            color = color_map.get(node_type)
        
        if color:
            node_data["itemStyle"] = {"color": color}
            
        # 僅為非葉節點遞迴，並按類型和名稱排序子節點
        children = sorted(
            [self._build_tree_data(graph, child_id) for child_id in graph.successors(node_id)],
            key=lambda x: (graph.nodes[x['name']].get('type', ''), x['name'])
        )
        if children:
            node_data["children"] = children
        return node_data

    def generate_mind_map_html(self, graph: nx.DiGraph):
        if not PYECHARTS_INSTALLED:
            logging.warning("Pyecharts 未安裝，跳過產生心智圖。")
            return
        if not graph or graph.number_of_nodes() <= 1:
            logging.warning("圖為空或節點太少，不產生心智圖。")
            return

        pyecharts_data = [self._build_tree_data(graph, self.config.ROOT_NODE_ID)]
        
        c = (
            Tree(init_opts=opts.InitOpts(width="100%", height="800px", theme="dark"))
            .add(series_name="", data=pyecharts_data, orient="LR", initial_tree_depth=2, symbol="circle", symbol_size=10)
            .set_global_opts(
                title_opts=opts.TitleOpts(title="104人力銀行_職位技能互動式心智圖", pos_left="center", title_textstyle_opts=opts.TextStyleOpts(color="#fff")),
                tooltip_opts=opts.TooltipOpts(trigger="item", trigger_on="mousemove", background_color="rgba(50,50,50,0.7)", border_width=1, textstyle_opts=opts.TextStyleOpts(color="#fff")),
                toolbox_opts=opts.ToolboxOpts(feature=opts.ToolBoxFeatureOpts(save_as_image=opts.ToolBoxFeatureSaveAsImageOpts(pixel_ratio=2, background_color="#222222")))
            )
            .set_series_opts(
                roam=True, 
                label_opts=opts.LabelOpts(position="right", vertical_align="middle", color="#ccc", font_size=12),
                layout="orthogonal", 
                edge_fork_position="60%", 
                edge_shape="curve"
            )
        )
        
        try:
            c.render(self.config.MIND_MAP_FILENAME)
            logging.info(f"成功產生心智圖檔案：{self.config.MIND_MAP_FILENAME}")
        except Exception as e:
            logging.error(f"產生心智圖失敗: {e}")


# --- 主程式執行區 ---
def main():
    """主程式進入點，作為協調者 (Orchestrator)，串連各個模組。"""
    try:
        config = Config()
        logging.info("程式開始執行...")

        # 步驟 1: 資料處理
        data_processor = DataProcessor(config)
        processed_df = data_processor.process()
        if processed_df is None:
            raise RuntimeError("資料處理失敗，程式中止。")

        # 步驟 2: 模型建立
        graph_builder = GraphBuilder(config)
        graph, entity_data = graph_builder.build(processed_df)
        if graph is None:
            raise RuntimeError("圖模型建立失敗，程式中止。")

        # 步驟 3: 報告生成
        logging.info("階段 3: 開始產生視覺化報告。")
        report_generator = ReportGenerator(config)
        report_generator.generate_network_html(graph, entity_data)
        report_generator.generate_mind_map_html(graph)
        
        logging.info("程式執行完畢。")

    except (FileNotFoundError, RuntimeError) as e:
        logging.error(f"程式執行失敗：{e}")
    except Exception as e:
        logging.critical(f"發生未預期的嚴重錯誤：{e}")
        logging.critical(traceback.format_exc())

if __name__ == "__main__":
    main()

2025-06-09 23:56:11,947 - INFO - 程式開始執行...
2025-06-09 23:56:11,948 - INFO - 正在讀取檔案：104人力銀行_職業探索_(20250609).csv
2025-06-09 23:56:12,019 - INFO - 階段 1: 完成檔案資料讀取與預處理。
2025-06-09 23:56:12,029 - INFO - 階段 2: 完成關係圖 Graph 建立。
2025-06-09 23:56:12,030 - INFO - 階段 3: 開始產生視覺化報告。
2025-06-09 23:56:12,051 - INFO - 成功產生網絡圖檔案：job_skill_network_20250609.html
2025-06-09 23:56:12,066 - INFO - 成功產生心智圖檔案：job_skill_mind_map_20250609.html
2025-06-09 23:56:12,068 - INFO - 程式執行完畢。
